In [5]:
# 1.- Funciones complementarias

# En este caso práctico relacionado con la detección de correos electrónicos de SPAM, 
# el conjunto de datos que disponemos esta formado por correos electrónicos, con sus 
# correspondientes cabeceras y campos adicionales. Por lo tanto, requieren un 
# preprocesamiento previo a que sean ingeridos por el algoritmo de Machine Learning.

In [7]:
# Esta clase facilita el preprocesamiento de correos electrónicos que poseen código HTML

from html.parser import HTMLParser

class MLStripper(HTMLParser):
  def __init__(self):
    self.reset()
    self.stict = False
    self.convert_charrefs = True
    self.fed = []

  def handle_data(self, d):
    self.fed.append(d)

  def get_data(self):
    return ''.join(self.fed)

In [9]:
# Esta función se encarga de elimar los tags HTML que se encuentren en el texto del correo electrónico

def strip_tags(html):

  s = MLStripper()

  s.feed(html)

  return s.get_data()

In [11]:
# Además de eliminar los posibles tags HTML que se encuentren en el correo electrónico, 
# deben realizarse otras acciones de preprocesamiento para evitar que los mensajes contengan
# ruido innecesario. Entre ellas se encuentra la eliminación de los signos de puntuación, 
# eliminación de posibles campos del correo electrónico que no son relevantes o eliminación
# de los afijos de una palabra manteniendo únicamente la raiz de la misma (Stemming). La 
# clase que se muestra a continuación realiza estas transformaciones.

In [13]:
!pip install nltk

In [14]:
import email
import string
import nltk
# nltk.download('stopwords')   ?????????

class Parser:

  def __init__(self):
    self.stemmer = nltk.PorterStemmer()
    self.stopwords = set(nltk.corpus.stopwords.words('english'))
    self.punctuation = list(string.punctuation)


  def parse(self, email_path):
    """Parse an email."""
    with open(email_path, errors='ignore') as e:
      msg = email.message_from_file(e)
    return None if not msg else self.get_email_content(msg)


  def get_email_content(self, msg):
    """Extract the email content."""
    subject = self.tokenize(msg['Subject']) if msg['Subject'] else []
    body = self.get_email_body(msg.get_payload(),
                  msg.get_content_type())
    content_type = msg.get_content_type()

    # Returning the content of the email
    return {"subject": subject,
        "body": body,
        "content_type": content_type}

  def get_email_body(self, payload, content_type):
    """Extract the body of the email."""
    body = []
    if type(payload) is str and content_type == 'text/plain':
      return self.tokenize(payload)
    elif type(payload) is str and content_type == 'text/html':
      return self.tokenize(strip_tags(payload))
    elif type(payload) is list:
      for p in payload:
        body += self.get_email_body(p.get_payload(),
                      p.get_content_type())
    return body

  def tokenize(self, text):
    """Transform a text string in tokens. Perform two main actions,
    clean the punctuation symbols and do stemming of the text."""
    for c in self.punctuation:
      text = text.replace(c, "")
    text = text.replace("\t", " ")
    text = text.replace("\n", " ")
    tokens = list(filter(None, text.split(" ")))
    # Stemming of the tokens
    return [self.stemmer.stem(w) for w in tokens if w not in self.stopwords]


#Lectura de un correo en formato raw
inmail = open("C:/Users/David/___utad/4_Cuarto/Q1/Aprendizaje_Automatico/datasets/trec07p/data/inmail.1").read()
print(inmail)

#Parsing del correo electrónico

p = Parser()
p.parse("C:/Users/David/___utad/4_Cuarto/Q1/Aprendizaje_Automatico/datasets/trec07p/data/inmail.1")


# Estas funciones complementarias se encargan cargar en memoria la ruta de cada correo electrónico y su etiqueta correspondiente {spam, ham}

index = open("C:/Users/David/___utad/4_Cuarto/Q1/Aprendizaje_Automatico/datasets/trec07p/full/index").readlines()

import os

DATASET_PATH = "C:/Users/David/___utad/4_Cuarto/Q1/Aprendizaje_Automatico/datasets/trec07p"


def parse_index(path_to_index, n_elements):
  ret_indexes = []
  index = open(path_to_index).readlines()
  for i in range(n_elements):
    mail = index[i].split(" ../")
    label = mail[0]
    path = mail[1][:-1]
    ret_indexes.append({"label":label, "email_path":os.path.join(DATASET_PATH, path)})
  return ret_indexes


def parse_email(index):
  p = Parser()
  pmail = p.parse(index["email_path"])
  return pmail, index["label"]

indexes = parse_index("C:/Users/David/___utad/4_Cuarto/Q1/Aprendizaje_Automatico/datasets/trec07p/full/index", 10)



From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="

In [15]:
# 2.- Procesamiento del conjunto de datos
# Cargamos el índice y las etiquetas en memoria
index = parse_index("C:/Users/David/___utad/4_Cuarto/Q1/Aprendizaje_Automatico/datasets/trec07p/full/index", 1)

# Leemos el primer correo
import os
open(index[0]["email_path"]).read()

# Parseamos el primer correo
mail, label = parse_email(index[0])
print("El correo es:", label)
print(mail)

El correo es: spam
{'subject': ['gener', 'ciali', 'brand', 'qualiti'], 'body': ['do', 'feel', 'pressur', 'perform', 'rise', 'occas', 'tri', 'viagra', 'anxieti', 'thing', 'past', 'back', 'old', 'self'], 'content_type': 'multipart/alternative'}


In [16]:
# El algoritmo de Regresión Logística no es capaz de ingerir texto como parte del conjunto de datos. 
# Por lo tanto, deben aplicarse una serie de funciones adicionales que transformen el texto de los correos 
# electrónicos parseados en una representación numérica.

In [17]:

#Aplicación de CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer

# Preapración del email en una cadena de texto
prep_email = [" ".join(mail['subject']) + " ".join(mail['body'])]
vectorizer = CountVectorizer()
X = vectorizer.fit(prep_email)

print("Email:", prep_email, "\n")
print("Características de entrada:", vectorizer.get_feature_names_out())

X = vectorizer.transform(prep_email)

print("\nValues:\n", X.toarray())

# Aplicación de OneHotEncoding
from sklearn.preprocessing import OneHotEncoder

prep_email = [[w] for w in mail['subject'] + mail['body']]
enc = OneHotEncoder(handle_unknown='ignore')
X = enc.fit_transform(prep_email)

print("Features:\n", enc.get_feature_names_out())
print("\nValues:\n", X.toarray())

# Funciones auxiliares para preprocesamiento del conjunto de datos
def create_prep_dataset(index_path, n_elements):
  X = []
  y = []
  indexes = parse_index(index_path, n_elements)
  for i in range(n_elements):
    print("\rParsing email: {0}".format(i+1), end='')
    mail, label = parse_email(indexes[i])
    X.append(" ".join(mail['subject']) + " ".join(mail['body']))
    y.append(label)
  return X, y

Email: ['gener ciali brand qualitido feel pressur perform rise occas tri viagra anxieti thing past back old self'] 

Características de entrada: ['anxieti' 'back' 'brand' 'ciali' 'feel' 'gener' 'occas' 'old' 'past'
 'perform' 'pressur' 'qualitido' 'rise' 'self' 'thing' 'tri' 'viagra']

Values:
 [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
Features:
 ['x0_anxieti' 'x0_back' 'x0_brand' 'x0_ciali' 'x0_do' 'x0_feel' 'x0_gener'
 'x0_occas' 'x0_old' 'x0_past' 'x0_perform' 'x0_pressur' 'x0_qualiti'
 'x0_rise' 'x0_self' 'x0_thing' 'x0_tri' 'x0_viagra']

Values:
 [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 

In [18]:

# 3.- Entrenamiento del algoritmo

# Leemos únicamente un subconjunto de 100 correos electrónicos
X_train, y_train = create_prep_dataset("C:/Users/David/___utad/4_Cuarto/Q1/Aprendizaje_Automatico/datasets/trec07p/full/index", 100)

# Aplicamos la sectorización a los datos
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
print(X_train.toarray())
print("\nFeatures:", len(vectorizer.get_feature_names_out()))
import pandas as pd
pd.DataFrame(X_train.toarray(), columns=[vectorizer.get_feature_names_out()])

# Entrenamiento del algoritmo de regresión logística con el conjunto de datos preprocesado (ver en sklearn logistic)
from sklearn.linear_model import LogisticRegression
# TODO ejecuta .fit() con los parámetros de entrada X_train e y_train
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)


Parsing email: 100[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Features: 4987


LogisticRegression()

In [21]:
# 4.- Predicción

#Lectura de un conjunto de correos nuevos:
# Leemos 150 correos de nuestro conjunto de datos y nos quedamos únicamente con los 50 últimos 
# Estos 50 correos electrónicos no se han utilizado para entrenar el algoritmo
X, y = create_prep_dataset("C:/Users/David/___utad/4_Cuarto/Q1/Aprendizaje_Automatico/datasets/trec07p/full/index", 150)
X_test = X[100:]
y_test = y[100:]

#TODO Preprocesalos con el vectorizador creado anteriormente
X_test = vectorizer.transform(X_test)
#TODO Predice el tipo de correo para estos correos de test
y_pred = log_reg.predict(X_test)
#Evalúa los resultados
from sklearn.metrics import accuracy_score
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))

Parsing email: 150Accuracy: 0.940


In [27]:
# 5.- Aumenta el conjunto de datos

# TODO Repite el proceso leyendo 12k correos electrónicos, 
# de los cuales utiliza 10k para entrenar el algoritmo y 2k para probarlo.

X, y = create_prep_dataset("C:/Users/David/___utad/4_Cuarto/Q1/Aprendizaje_Automatico/datasets/trec07p/full/index", 12000)

X_train = X[:10000]
y_train = y[:10000]
X_test = X[10000:]
y_test = y[10000:]

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

log_reg = LogisticRegression(max_iter=1000)  
log_reg.fit(X_train, y_train)

X_test = vectorizer.transform(X_test)

y_pred = log_reg.predict(X_test)

# Evalúa los resultados
from sklearn.metrics import accuracy_score
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))

Parsing email: 12000Accuracy: 0.987
